**Test**: Equal Number of Real News and Fake News from Global South, and checking PRA values.

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
!pip install keras
!pip install tensorflo

ERROR: Could not find a version that satisfies the requirement tensorflo (from versions: none)
ERROR: No matching distribution found for tensorflo


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import tensorflow as tf
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM, Dropout
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.callbacks import History

from keras.preprocessing.text import Tokenizer
from keras import layers
from keras.layers import Embedding
from keras.layers import Reshape, Flatten
from keras.models import Sequential, Model
from keras.layers import Dense, Activation
from keras.layers import Flatten, Conv1D, SpatialDropout1D, MaxPooling1D
from keras.layers import AveragePooling1D, Input, Dropout, concatenate

from wordcloud import WordCloud, STOPWORDS

import pickle
from keras.callbacks import EarlyStopping
# from sklearn import resample
from imblearn.over_sampling import SMOTE

### read data

In [5]:
# For IND
df_ind = pd.read_csv('/content/drive/MyDrive/Datasets_BTP/IND/IND_poli.csv')
df_ind['label'] = 1
# df_ind.columns
df_ind = df_ind[['News Abstract','label']]
df_ind.columns = ['text', 'label']
df_ind.shape

# df = df_ind

(200, 2)

In [6]:
# For FNI
df_fni = pd.read_csv('/content/drive/MyDrive/Datasets_BTP/FNI/fakenewsincidents_india_dataset_v1.csv')
df_fni['label'] = 0
df_fni = df_fni[['Content','label']]
df_fni.columns = ['text', 'label']
# df_fni = df_fni_full.sample(868)
df_fni.shape

(4802, 2)

In [7]:
df_toi_1 = pd.read_csv('/content/drive/MyDrive/Datasets_BTP/TOI_Scrapped/12-02-2024-Articles.csv')
df_toi_2 = pd.read_csv('/content/drive/MyDrive/Datasets_BTP/TOI_Scrapped/13-02-2024-Articles.csv')
df_toi_3 = pd.read_csv('/content/drive/MyDrive/Datasets_BTP/TOI_Scrapped/14-02-2024-Articles.csv')
df_toi_4 = pd.read_csv('/content/drive/MyDrive/Datasets_BTP/TOI_Scrapped/12-02-2024-Articles.csv')
df_toi_5 = pd.read_csv('/content/drive/MyDrive/Datasets_BTP/TOI_Scrapped/16-02-2024-Articles.csv')
df_toi_6 = pd.read_csv('/content/drive/MyDrive/Datasets_BTP/TOI_Scrapped/20-02-2024-Articles.csv')
df_toi_7 = pd.read_csv('/content/drive/MyDrive/Datasets_BTP/TOI_Scrapped/24-02-2024-Articles.csv')
df_toi_8 = pd.read_csv('/content/drive/MyDrive/Datasets_BTP/TOI_Scrapped/9-02-2024-Articles.csv')
df_toi = pd.concat([df_toi_1, df_toi_2, df_toi_3, df_toi_4, df_toi_5, df_toi_6, df_toi_7, df_toi_8])

df_toi['label'] = 1
df_toi = df_toi[['text', 'label']]

df_toi.shape

(668, 2)

In [8]:
# df_fni_checking = pd.read_excel('/content/drive/MyDrive/Datasets_BTP/FNI/fakenewsincidents_india_dataset_v1.xls')
# df_fni_checking.columns
df_isot = pd.read_csv('/content/drive/MyDrive/Datasets_BTP/ISOT/True_GS.csv', delimiter = ',')
df_isot['label'] = 1
df_isot = df_isot[['text','label']]
df_isot.shape

(4873, 2)

In [9]:
# FOR DATASETS COMBINED
# df = pd.concat([df_isot, df_fni, df_ind])
df = pd.concat([df_fni, df_isot, df_ind, df_toi])
# df = pd.concat([df_isot, df_ind, df_toi])

In [10]:
df.to_csv('True_GS_complete.csv')

In [ ]:
# Maximum number of words to be embedded
NUM_WORDS = 30000
EMBEDDING_DIM=100
# max length to be encoded for a sentence
MAX_SEQUENCE_LENGTH = 1000

df['text'] = df['text'].astype(str)

# Define Tokenize text function
tokenizer = Tokenizer(num_words=NUM_WORDS,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'', lower=True)

# Fit the function on the text
tokenizer.fit_on_texts(df['text'])

sequences = tokenizer.texts_to_sequences(df['text'])

# pad the data to the same length
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH,padding='post')

# Count number of unique tokens
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 71815 unique tokens.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, df[['label']], test_size=0.2,  random_state=42)

In [ ]:
num_class_A_train = np.sum(y_train == 0)
num_class_B_train = np.sum(y_train == 1)
num_class_A_test = np.sum(y_test == 0)
num_class_B_test = np.sum(y_test == 1)
print(num_class_A_train)
print(num_class_A_test)
print(num_class_B_train)
print(num_class_B_test)

label    3825
dtype: int64
label    977
dtype: int64
label    4609
dtype: int64
label    1132
dtype: int64


In [ ]:
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

num_class_A_train = np.sum(y_train == 0)
num_class_B_train = np.sum(y_train == 1)
num_class_A_test = np.sum(y_test == 0)
num_class_B_test = np.sum(y_test == 1)
print(num_class_A_train)
print(num_class_A_test)
print(num_class_B_train)
print(num_class_B_test)

label    4609
dtype: int64
label    977
dtype: int64
label    4609
dtype: int64
label    1132
dtype: int64


### glove embedding

In [ ]:
word_vectors = dict()

# load the whole embedding into memory
f = open('/content/drive/MyDrive/glove.6B.100d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_vectors[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(word_vectors))

Loaded 400000 word vectors.


In [ ]:
vocabulary_size=min(len(word_index)+1,(NUM_WORDS))
#vocabulary_size= len(word_index)+1
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))

for word, i in word_index.items():
    if i>=NUM_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
      # if the word is not found, set to all 0s
        vec = np.zeros(EMBEDDING_DIM)
        embedding_matrix[i]=vec

#del(word_vectors)

In [ ]:
# Dfine Embedding function using the embedding_matrix
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [ ]:
embedding_matrix.shape

(30000, 100)

In [ ]:
X_train.shape

(9218, 1000)

### FNDNet

In [ ]:
num_kernels = 128
kernel_sizes= [3,4,5]
pool_sizes = [800,799,798]

#embed_input = Input(shape=(max_length,))
#x = Embedding(vocab_size,output_dim,input_length=max_length)(embed_input)

inputs = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='float32')
embed_input = embedding_layer(inputs)

pool_outputs = []
for i in range(len(kernel_sizes)):
    conv = Conv1D(num_kernels, kernel_size=kernel_sizes[i], padding='valid', activation='relu')(embed_input)
    pool = MaxPooling1D(pool_size=pool_sizes[i], strides=1, padding = 'valid')(conv)
    pool_outputs.append(pool)
merge = concatenate(pool_outputs,axis = 1)

conv = Conv1D(num_kernels, kernel_size=5, activation='relu')(merge)
pool = MaxPooling1D(pool_size=5)(conv)
conv = Conv1D(num_kernels, kernel_size=5, activation='relu')(pool)
pool = MaxPooling1D(pool_size=30)(conv)

x = Flatten()(pool)
#x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(1, activation='sigmoid')(x)
modelC = Model(inputs=inputs,outputs=x)
modelC.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1000)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, 1000, 100)            3000000   ['input_1[0][0]']             
                                                                                                  
 conv1d (Conv1D)             (None, 998, 128)             38528     ['embedding[0][0]']           
                                                                                                  
 conv1d_1 (Conv1D)           (None, 997, 128)             51328     ['embedding[0][0]']           
                                                                                              

In [ ]:
modelC.compile(loss='binary_crossentropy',
              #optimizer='Adadelta',
              optimizer='Adam',
              metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [ ]:
epochs = 40
batch_size = 128
xtrain, xvald, ytrain, yvald = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

early_stopping = EarlyStopping(monitor='val_loss',  # Monitor validation loss
                               patience=3,         # Number of epochs with no improvement after which training will be stopped
                               verbose=1,          # Will print messages when early stopping triggers
                               mode='min',         # The direction is automatically inferred if not set but can be set to 'min' or 'max'
                               restore_best_weights=True)  # Whether to restore model weights from the epoch with the best value of the monitored quantity.

# Fit the model with the early stopping callback
modelC.fit(xtrain, ytrain,
           epochs=epochs,
           validation_data=(xvald, yvald),
           batch_size=batch_size,
           verbose=1,
           callbacks=[early_stopping])

Epoch 1/40
58/58 [==============================] - 18s 175ms/step - loss: 0.3087 - precision: 0.8510 - recall: 0.8384 - val_loss: 0.0511 - val_precision: 0.9738 - val_recall: 0.9889
Epoch 2/40
58/58 [==============================] - 8s 141ms/step - loss: 0.0250 - precision: 0.9922 - recall: 0.9908 - val_loss: 0.0433 - val_precision: 0.9741 - val_recall: 0.9978
Epoch 3/40
58/58 [==============================] - 8s 141ms/step - loss: 0.0059 - precision: 0.9981 - recall: 0.9989 - val_loss: 0.0146 - val_precision: 0.9967 - val_recall: 0.9922
Epoch 4/40
58/58 [==============================] - 8s 143ms/step - loss: 5.6154e-04 - precision: 0.9997 - recall: 1.0000 - val_loss: 0.0155 - val_precision: 0.9967 - val_recall: 0.9900
Epoch 5/40
58/58 [==============================] - 8s 142ms/step - loss: 3.3224e-04 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0144 - val_precision: 0.9967 - val_recall: 0.9922
Epoch 6/40
58/58 [==============================] - 8s 143ms/step - loss: 1.9426e

In [ ]:
loss, pr, recall = modelC.evaluate(X_test, y_test)
print(pr)
print(recall)
print((2*pr*recall)/(pr+recall))

66/66 [==============================] - 2s 18ms/step - loss: 0.0148 - precision: 0.9956 - recall: 0.9947
0.9955791234970093
0.9946996569633484
0.9951391959203687


In [ ]:
# Save the trained model as a pickle string.
saved_model_path = 'FNDNet_GS_Model.pkl'
with open(saved_model_path, 'wb') as file:
    pickle.dump(modelC, file)

with open('tokenizer_GS_model.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)